# MLP

#### Import libraries:

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import random
%matplotlib notebook
! conda env list

# conda environments:
#
base                     /Users/juandanielgomez/opt/anaconda3
Vision2               *  /Users/juandanielgomez/opt/anaconda3/envs/Vision2
env27                    /Users/juandanielgomez/opt/anaconda3/envs/env27



## I. Construct data set
<p> In orther to do this we'll parse each of the annotation files to extract the boxes boundaries on each image from folder 1 to 8:  </p>

#### Parse annotation file:
Open file, read name and store all the boxes of each image in a dictionary.
The dictionary key is the image name, for each image there is a list of faces:

In [4]:
def parse_annotation_file(annotation_file_name):
    fd = open(annotation_file_name, 'r')
    dict_boxes = {}
    while True:
        img_name = fd.readline().rstrip()
        if not img_name:
            break
        nb_faces = int(fd.readline().rstrip())
        dict_boxes[img_name] = []
        for i in range(nb_faces):
            ellipse = fd.readline().rstrip().split()
            major_axis, minor_axis, angle, center_x, center_y, _ = ellipse
            x1 = float(center_x) - float(minor_axis)
            y1 = float(center_y) - float(major_axis)
            x2 = float(center_x) + float(minor_axis)
            y2 = float(center_y) + float(major_axis)
            if (y1 <0):
                y1 = 0
            if (x1 < 0):
                x1 = 0
            box = [int(x1),int(y1),int(x2),int(y2)]
            dict_boxes[img_name].append(box)
    fd.close()
    return dict_boxes

#### Let's crop the imagettes and save it on another file:

In [15]:
def extract_imagettes(dict_boxes,i):
    for j, img_name in enumerate(dict_boxes.keys()):
        img = cv2.imread('../originalPics/'+img_name+'.jpg')
        list_faces = dict_boxes[img_name]
        if (i == 0) & (j ==11):
            print(img_name)
        for k, box in enumerate(list_faces):
            x1,y1,x2,y2 = box
            img_cropped_positive = img[y1:y2,x1:x2]
            cv2.imwrite('./TRAINSET/Positive/Folder'+str(i+1)+'-'+str(j)+'-'+str(k)+'.jpg',img_cropped_positive)
            img_cropped_negative = true_negative_imagettes_mp(box,list_faces, img,i,j,k)
            try:
                cv2.imwrite('./TRAINSET/Negative/Folder'+str(i+1)+'-'+str(j)+'-'+str(k)+'.jpg',img_cropped_negative)
            except:
                pass

#### Get a true negative:

In [16]:
def true_negative_imagettes_mp(current_face,list_faces,img,i,j,k):
    x1,y1,x2,y2 = current_face
    width = x2 - x1 + 1
    height = y2 - y1 + 1
    area_of_negative = width * height
    y_lim, x_lim , _ = img.shape
    IOU = 1
    while(IOU >= 0.001):
        list_of_IOU = []
        x2_negative = random.randint(0,x_lim - 1 )
        y2_negative = random.randint(0, y_lim - 1 )
        x1_negative = x2_negative - width
        y1_negative = y2_negative - height 
        if(x1_negative < 0):
            x1_negative = 0
        if(y1_negative < 0):
            y1_negative = 0
        for box in list_faces:
            x1,y1,x2,y2 = box
            width = x2 - x1 + 1
            height = y2 - y1 + 1
            area = width * height
            ## intersection:
            x1_i=max(x1,x1_negative)
            x2_i=min(x2,x2_negative)
            y1_i=max(y1,y1_negative)
            y2_i=min(y2,y2_negative)
            ## area of intersection:
            area_intersection_box= (x2_i - x1_i + 1) * (y2_i - y1_i +1)
            if (area_of_negative + area - area_intersection_box) == 0:
                IOU = area_intersection_box / (area_of_negative + area + 1 - area_intersection_box)
            else:
                IOU = area_intersection_box / (area_of_negative + area - area_intersection_box)
            if(IOU <= 0.001):
                list_of_IOU.append(IOU)
                continue
            else:
                break
    print('IOU of '+str(i+1)+'/'+str(j)+'-'+str(k)+'.jpg is: '+str(list_of_IOU))
    img_cropped_negative = img[y1_negative:y2_negative,x1_negative:x2_negative]
    return img_cropped_negative

#### Extract  imagettes (Run):

In [17]:
for i in range(8):
    dict_boxes= parse_annotation_file('../FDDB-folds/FDDB-fold-0'+str(i+1)+'-ellipseList.txt')
    extract_imagettes(dict_boxes,i)

IOU of 1/0-0.jpg is: [-0.34293504212942283]
IOU of 1/1-0.jpg is: [-0.03773584905660377, -0.06376266120551281, -0.005209585637573135]
IOU of 1/1-1.jpg is: [-0.17249645320679027, -0.41968718967229396, -0.5807564311220152]
IOU of 1/1-2.jpg is: [-0.15120016412500856, 0.0005797437532610586, -0.035856573705179286]
IOU of 1/2-0.jpg is: [-0.4198679555394601]
IOU of 1/3-0.jpg is: [-0.026730637422892393]
IOU of 1/4-0.jpg is: [-2.8673535093367675, -0.166734378808808]
IOU of 1/4-1.jpg is: [-0.2580668499493561, -0.05830249701349028]
IOU of 1/5-0.jpg is: [-0.18801179168031446, -0.40852575488454707]
IOU of 1/5-1.jpg is: [-0.014971325083006339, -0.03236351748937561]
IOU of 1/6-0.jpg is: [-1.4866964916411585, -0.11310500990659496, -1.1730493058879847]
IOU of 1/6-1.jpg is: [-6.415485655029381, -0.0007714632085522208, -1.8702865761689291]
IOU of 1/6-2.jpg is: [-1.2413821884650698, 0.0, -1.0668422118695564]
IOU of 1/7-0.jpg is: [-0.005601002284619353, -0.0702288836085217]
IOU of 1/7-1.jpg is: [-0.03523486

IOU of 1/92-0.jpg is: [-0.19954519613416713]
IOU of 1/93-0.jpg is: [-0.008281494485364272]
IOU of 1/94-0.jpg is: [-0.1637084986427229, -0.42193503318197695]
IOU of 1/94-1.jpg is: [-0.17224530624807632, -0.5757697546162949]
IOU of 1/95-0.jpg is: [-0.22490717205673924]
IOU of 1/96-0.jpg is: [-2.059366754617414, -4.383892617449664]
IOU of 1/96-1.jpg is: [-5.798248953178531, -6.743421052631579]
IOU of 1/97-0.jpg is: [-0.32233544660935376]
IOU of 1/98-0.jpg is: [-0.22505922611213477]
IOU of 1/99-0.jpg is: [-0.031945788964181994, -0.23884694577899795, -0.4808486669170109, -0.2884347422201579]
IOU of 1/99-1.jpg is: [-0.022169038921778562, -0.21923076923076923, -0.4332846715328467, -0.3046547734130123]
IOU of 1/99-2.jpg is: [-0.0036833887176202108, -0.04242723236309817, -0.07515751575157516, -0.31274246493584007]
IOU of 1/99-3.jpg is: [-0.008322544849269465, -0.0224043490795272, 0.00023690133059580683, -0.13379223039797453]
IOU of 1/100-0.jpg is: [-0.08432474833311544]
IOU of 1/101-0.jpg is: [

IOU of 1/178-0.jpg is: [-0.003575685339690107]
IOU of 1/179-0.jpg is: [-0.10265165749015635]
IOU of 1/180-0.jpg is: [-0.011824990145841546]
IOU of 1/181-0.jpg is: [-0.06627446272061471]
IOU of 1/182-0.jpg is: [-0.2955536181342633, -0.023184868822452714, -0.0796478725634039]
IOU of 1/182-1.jpg is: [-0.22474426487500757, -0.562140645731977, -0.6942800788954635]
IOU of 1/182-2.jpg is: [-0.024294496703947857, -0.09544212159267193, -0.017759775665991587]
IOU of 1/183-0.jpg is: [-0.11100196463654224]
IOU of 1/184-0.jpg is: [0.00068300586114332]
IOU of 1/185-0.jpg is: [-0.05087644292432664, -0.06686083322780803]
IOU of 1/185-1.jpg is: [0.0, -0.023952934584675727]
IOU of 1/186-0.jpg is: [-0.03873395307658256]
IOU of 1/187-0.jpg is: [-0.11850895594642569, -0.09198778195488722]
IOU of 1/187-1.jpg is: [-0.10426979685766792, -0.1662257632815661]
IOU of 1/188-0.jpg is: [-0.07099730850367353]
IOU of 1/189-0.jpg is: [-0.001327433628318584, -0.08395562447811046]
IOU of 1/189-1.jpg is: [-0.297188460975

IOU of 1/262-0.jpg is: [-0.05192834312553518]
IOU of 1/263-0.jpg is: [-0.001062497628353508]
IOU of 1/264-0.jpg is: [-0.005370802365323062, -0.06509183319856764, -0.4228892842794008, -0.12274832732887288, 0.0, -0.24133296438312843, -0.2116525955028003, -0.027574681428869856, -0.053577183370931356, -0.07106690777576853]
IOU of 1/264-1.jpg is: [-0.0023420647149460707, -0.015462290943515304, -0.41173845123856284, -0.21766072811773818, -0.04608441730820281, -0.24689111313315135, -0.22279129321382843, -0.0856522849119224, -0.12520503007107708, -0.09900221729490022]
IOU of 1/264-2.jpg is: [-0.09332732191163211, -0.008525576730190571, -1.6646706586826348, -1.2303689687795647, -2.143114852675887, -1.7634782608695652, -1.30622009569378, -2.5411061285500747, -1.6262188515709644, -2.309055118110236]
IOU of 1/264-3.jpg is: [-0.13220195163343232, -0.0039447731755424065, -2.581168831168831, -1.3158522050059596, -3.290530303030303, -3.0467353951890033, -1.5165821303160358, -7.573770491803279, -1.9154

IOU of 2/46-0.jpg is: [-0.08553554093409557, -0.029168519341929745]
IOU of 2/46-1.jpg is: [-0.12838323353293413, -0.013861386138613862]
IOU of 2/47-0.jpg is: [-0.4041411042944785, -0.28047783164904494]
IOU of 2/47-1.jpg is: [-0.14268313902905863, -0.0856539660199055]
IOU of 2/48-0.jpg is: [-4.0688524590163935, -0.27156726768377254]
IOU of 2/48-1.jpg is: [-0.007558578987150416, -0.3668114026807627]
IOU of 2/49-0.jpg is: [-0.14049586776859505]
IOU of 2/50-0.jpg is: [-0.006365248408687898]
IOU of 2/51-0.jpg is: [-0.09649468294604174, -0.012851810753555936, -0.315886352165781]
IOU of 2/51-1.jpg is: [-0.2549640638323791, -0.0015368065160596281, -0.06819647259624502]
IOU of 2/51-2.jpg is: [-0.2583998455200824, 0.0, -0.06924561685700499]
IOU of 2/52-0.jpg is: [-0.26043317485472794]
IOU of 2/53-0.jpg is: [-0.014500127194098194, -0.3909651319591601, -0.4262043091993851, -0.6107522625452059]
IOU of 2/53-1.jpg is: [-0.2257155977345783, -0.002545565624681804, -70.16949152542372, -0.101803833145434

...

IOU of 7/268-0.jpg is: [-0.023109243697478993]
IOU of 7/269-0.jpg is: [-0.10422009792492422]
IOU of 7/270-0.jpg is: [-0.025881192999753513]
IOU of 7/271-0.jpg is: [-0.05401773716742811, -0.19645905283633044]
IOU of 7/271-1.jpg is: [-0.055259964529076826, -1.295156027775454]
IOU of 7/272-0.jpg is: [-0.025309005297233667]
IOU of 7/273-0.jpg is: [-0.015105999485022744, -0.08568402055788772]
IOU of 7/273-1.jpg is: [-0.014796026210103572, -0.30483325987953575]
IOU of 7/274-0.jpg is: [-0.3181723346547048]
IOU of 7/275-0.jpg is: [-0.22218329245707993]
IOU of 7/276-0.jpg is: [-0.15546582059417363]
IOU of 7/277-0.jpg is: [-0.07110826393337603]
IOU of 8/0-0.jpg is: [-0.3267530684061454, -0.4952192045570135]
IOU of 8/0-1.jpg is: [-0.19285975684137763, -0.40381368556957]
IOU of 8/1-0.jpg is: [-0.17979748085947148, -0.47955912950570767, -0.6313538923342679]
IOU of 8/1-1.jpg is: [-0.061168594035301276, -0.5092584618034622, -0.536920252255129]
IOU of 8/1-2.jpg is: [-0.18530571374929594, 0.0, -2.45130

IOU of 8/78-0.jpg is: [-0.008570624504310896, -0.022815827308149104]
IOU of 8/78-1.jpg is: [-0.07036948937519823, 0.00025440668726149374]
IOU of 8/79-0.jpg is: [-0.045771916214119475]
IOU of 8/80-0.jpg is: [-0.015836705965159247]
IOU of 8/81-0.jpg is: [-0.001122255734025391]
IOU of 8/82-0.jpg is: [-0.34372969460688757, -0.4137580481192816]
IOU of 8/82-1.jpg is: [-0.3954571927781013, -0.20748486516235554]
IOU of 8/83-0.jpg is: [-0.4214046822742475]
IOU of 8/84-0.jpg is: [-0.02337728843938036, -0.35635677601928517, -0.1928959249020505]
IOU of 8/84-1.jpg is: [-0.02206702004231408, -0.7061855670103093, -0.5616199301961728]
IOU of 8/84-2.jpg is: [-0.08173055165859619, -0.053035226829585336, -0.17525773195876287]
IOU of 8/85-0.jpg is: [-0.08435481251015231, -0.33528870485392226]
IOU of 8/85-1.jpg is: [-0.05769492078158545, -0.3576149185431049]
IOU of 8/86-0.jpg is: [-0.04562883435582822, -0.2768595041322314]
IOU of 8/86-1.jpg is: [-0.0475415174210355, -0.07172995780590717]
IOU of 8/87-0.jpg 

IOU of 8/150-0.jpg is: [-5.235017626321974]
IOU of 8/151-0.jpg is: [-0.041112752338749385]
IOU of 8/152-0.jpg is: [-0.10937580803640688, -0.40871483728626584]
IOU of 8/152-1.jpg is: [-0.026752209175644595, -0.2251270878721859]
IOU of 8/153-0.jpg is: [-0.5602314571278275, -0.05021432945499082, -0.3700962250185048, -0.6992618762223204, -0.21174902408835572]
IOU of 8/153-1.jpg is: [-0.037243947858473, -0.1617044523354275, -0.021032951624211264, -0.08514157828159968, -0.2436171200445641]
IOU of 8/153-2.jpg is: [-0.32727272727272727, -0.04545972427936652, -0.08196721311475409, -0.4716836244960645, -0.049228015215931974]
IOU of 8/153-3.jpg is: [-0.20350877192982456, -0.44042375907630044, -2.3801404212637913, -0.0060901339829476245, -1.594945278122532]
IOU of 8/153-4.jpg is: [-0.4749159637216972, -0.12190127182582453, -0.5504655099449852, -0.7143398884013219, -0.26383290582631]
IOU of 8/154-0.jpg is: [0.000336823195890757]
IOU of 8/155-0.jpg is: [-0.3113816121613283]
IOU of 8/156-0.jpg is: [-

IOU of 8/217-0.jpg is: [-0.02184324261184441]
IOU of 8/218-0.jpg is: [-0.06980454726765058]
IOU of 8/219-0.jpg is: [-0.018019507356587867, -2.7053888351080992]
IOU of 8/219-1.jpg is: [0.0, -2.147517321016166]
IOU of 8/220-0.jpg is: [-0.02993854642591268]
IOU of 8/221-0.jpg is: [-0.12434970181449055, -0.4693526170798898, -0.04725342968441258, -0.37170921420023934]
IOU of 8/221-1.jpg is: [-0.11842105263157894, -0.47035573122529645, -0.04021979266980117, -0.49108147195886226]
IOU of 8/221-2.jpg is: [-0.016779305523188067, -0.011666569445254623, -0.09210885592063348, -0.007597340930674264]
IOU of 8/221-3.jpg is: [-0.013156864280679772, -0.13268519786389155, -0.26141450313339304, -0.011703511053315995]
IOU of 8/222-0.jpg is: [-0.046894803548795945, -0.012584704743465635, -0.43581883669246685]
IOU of 8/222-1.jpg is: [-0.00400351528171077, -0.5038311406679196, -0.16387665198237886]
IOU of 8/222-2.jpg is: [-0.08227374719521316, -0.3594330070874114, -0.11961934270222419]
IOU of 8/223-0.jpg is: 

IOU of 8/247-0.jpg is: [-0.034765333995530175, -0.08012085570152958]
IOU of 8/247-1.jpg is: [-0.08765217391304347, -0.2220276937768582]
IOU of 8/248-0.jpg is: [-0.18840579710144928, -0.008619141677141317]
IOU of 8/248-1.jpg is: [-0.016302895322939867, -0.3965476914440754]
IOU of 8/249-0.jpg is: [-0.08694209101231548]
IOU of 8/250-0.jpg is: [-0.028833008823894942]
IOU of 8/251-0.jpg is: [-0.18355021480605194, -0.012532956325704024]
IOU of 8/251-1.jpg is: [-0.024792452830188678, -0.04786836200448766]
IOU of 8/252-0.jpg is: [-0.01469842878864673]
IOU of 8/253-0.jpg is: [-0.10611561016475844]
IOU of 8/254-0.jpg is: [-0.08108935128518972]
IOU of 8/255-0.jpg is: [-0.03413498836307215]
IOU of 8/256-0.jpg is: [-0.3129591899734519]
IOU of 8/257-0.jpg is: [-0.09162909711222128, -0.2341925302472383]
IOU of 8/257-1.jpg is: [-0.11339829476248478, -0.0805523590333717]
IOU of 8/258-0.jpg is: [-0.09561812979972896]
IOU of 8/259-0.jpg is: [-0.05719237435008666]
IOU of 8/260-0.jpg is: [-0.11304347826086

## II. Construct and train a MLP model:

In [27]:
import keras
from keras import layers
from keras.models import Sequential
import tensorflow as tf
from glob import glob

 #### Let's begin with a 24x24 window:

In [28]:
img_window = (8,8,1)
window_size = np.prod(img_window)

#### Create the model:

In [26]:
MLP = Sequential(
    [
        layers.InputLayer(input_shape=img_window),
        layers.Flatten(),
        layers.Dense(window_size//2,activation="relu"),
        layers.Dense(window_size//2,activation="relu"),
        layers.Dense(2, activation = "softmax"),
    ]
)
MLP.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_17 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 66        
Total params: 3,202
Trainable params: 3,202
Non-trainable params: 0
_________________________________________________________________
